In [1]:

from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout
from Data_manager.Movielens.Movielens1MReader import Movielens1MReader
from Evaluation.Evaluator import EvaluatorHoldout

dataset_reader = Movielens1MReader()

dataSplitter = DataSplitter_Holdout(dataset_reader, user_wise=False, split_interaction_quota_list=[80, 10, 10])
dataSplitter.load_data() #"results_experiments/Movielens1M/data"
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

cutoff_list = [5, 10, 20, 30, 40, 50, 100]
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

DataSplitter_Holdout: Verifying data consistency...
DataSplitter_Holdout: Verifying data consistency... Passed!
DataSplitter_Holdout: DataReader: Movielens1M
	Num items: 3883
	Num users: 6040
	Train 		quota 80.00 (80.00), 	interactions 800167, 	density 3.41E-02
	Validation 	quota 10.00 (10.00), 	interactions 100021, 	density 4.26E-03
	Test 		quota 10.00 (10.00), 	interactions 100021, 	density 4.26E-03



	ICM name: ICM_genres, Num features: 18, feature occurrences: 6408, density 9.17E-02
	ICM name: ICM_year, Num features: 1, feature occurrences: 3883, density 1.00E+00
	UCM name: UCM_all, Num features: 3469, feature occurrences: 24160, density 1.15E-03


DataSplitter_Holdout: Done.
EvaluatorHoldout: Ignoring 75 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 80 ( 1.3%) Users that have less than 1 test interactions


In [2]:
!pip install optuna


In [3]:
import optuna
import numpy as np

def create_objective(recommender_class, URM_train, URM_validation, hyperparam_space, evaluator_validation):
    def objective(trial):
        # Gather and set hyperparameters
        hyperparams = {name: trial.suggest_categorical(name, space) for name, space in hyperparam_space.items()}

        # Initialize and train the recommender
        recommender = recommender_class(URM_train)
        recommender.fit(**hyperparams)

        # Evaluate recommender
        user_id_array = np.arange(URM_validation.shape[0])
        
        result_df, _ = evaluator_validation.evaluateRecommender(recommender)

        return result_df.loc[10]['MAP']

    return objective


In [4]:
import optuna
import numpy as np
import pandas as pd
import os

def create_objective_d(recommender_instance, URM_train, URM_validation, hyperparam_space, evaluator_validation):
    def objective(trial):
        # Gather and set hyperparameters
        hyperparams = {name: trial.suggest_categorical(name, space) for name, space in hyperparam_space.items()}

        # Initialize and train the recommender
        
        recommender_instance.fit(**hyperparams)

        # Evaluate recommender
        user_id_array = np.arange(URM_validation.shape[0])
        
        result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
        result_df['hyperparams'] = str(hyperparams)

        # Assuming df_row is the DataFrame containing the row to append

        filename = recommender_instance.RECOMMENDER_NAME + ".csv"

        # Check if file exists
        if os.path.isfile(filename):
            # If it exists, append without writing the header
           pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
            # If it doesn't exist, create it, write the header
           pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)
    
        return result_df.loc[10]['MAP']

    return objective


In [5]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

itemKNNinstance = ItemKNNCFRecommender(URM_train= URM_train)
itemKNNinstance.fit()
result_df, _ = evaluator_validation.evaluateRecommender(itemKNNinstance)
filename = "prova.csv"

# Check if file exists
if os.path.isfile(filename):
    # If it exists, append without writing the header
    pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
else:
    # If it doesn't exist, create it, write the header
    pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)
print(type(result_df.loc[10]))


ItemKNNCFRecommender: URM Detected 204 ( 5.3%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 3883 (100.0%), 4339.86 column/sec. Elapsed time 0.89 sec
EvaluatorHoldout: Processed 5965 (100.0%) in 2.14 sec. Users per second: 2783
<class 'pandas.core.series.Series'>


In [6]:
pd.DataFrame(result_df.loc[10]).transpose()

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
10,0.165968,0.214284,0.143382,0.087172,0.107448,0.385499,0.187772,0.153851,0.727075,0.5701,...,0.987583,0.718046,0.987583,0.071175,8.451121,0.995768,0.203534,0.781573,1.627008,0.066553


In [7]:

# Define the hyperparameter space
itemknn_hyperparam_space = {
    "topK": list(range(10, 100)),
    "shrink": list(range(10, 100)),
    "similarity": ["cosine", "jaccard"],
    "normalize": [True, False],
    "feature_weighting": ["BM25", "TF-IDF", "none"]
}

itemKNNinstance = ItemKNNCFRecommender(URM_train= URM_train)
# Create an objective function
objective = create_objective_d(recommender_instance=itemKNNinstance, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=itemknn_hyperparam_space, evaluator_validation=evaluator_validation)

# Create the study and run optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=150)


[I 2023-07-26 18:53:29,934] A new study created in memory with name: no-name-64d42b04-774d-4bd5-8c04-8da72449ceee


ItemKNNCFRecommender: URM Detected 204 ( 5.3%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 3883 (100.0%), 4799.30 column/sec. Elapsed time 0.81 sec
EvaluatorHoldout: Processed 5965 (100.0%) in 1.86 sec. Users per second: 3209


[I 2023-07-26 18:53:32,621] Trial 0 finished with value: 0.07928532843704722 and parameters: {'topK': 16, 'shrink': 94, 'similarity': 'jaccard', 'normalize': False, 'feature_weighting': 'none'}. Best is trial 0 with value: 0.07928532843704722.


Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 3883 (100.0%), 4451.20 column/sec. Elapsed time 0.87 sec
EvaluatorHoldout: Processed 5965 (100.0%) in 2.25 sec. Users per second: 2646


[I 2023-07-26 18:53:35,788] Trial 1 finished with value: 0.06177522984605995 and parameters: {'topK': 60, 'shrink': 58, 'similarity': 'cosine', 'normalize': False, 'feature_weighting': 'TF-IDF'}. Best is trial 0 with value: 0.07928532843704722.


...

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 3883 (100.0%), 4850.77 column/sec. Elapsed time 0.80 sec
EvaluatorHoldout: Processed 5965 (100.0%) in 1.99 sec. Users per second: 2991


[I 2023-07-26 18:55:08,102] Trial 32 finished with value: 0.08390182812437653 and parameters: {'topK': 23, 'shrink': 90, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'}. Best is trial 3 with value: 0.09149159781263762.


Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 3883 (100.0%), 4405.41 column/sec. Elapsed time 0.88 sec


[W 2023-07-26 18:55:10,546] Trial 33 failed with parameters: {'topK': 38, 'shrink': 18, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucaortolomo/opt/anaconda3/envs/tesi/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/vy/ssvybtks5ms1n3l5ldrh_hv40000gn/T/ipykernel_60550/3884821255.py", line 18, in objective
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
  File "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Evaluation/Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Evaluation/Evaluator.py", line 484, in _run_evaluation_on_selected_users
    resul

KeyboardInterrupt: 

'ItemKNNCFRecommender'

In [ ]:
{'topK': 96, 'shrink': 59, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'}

In [ ]:
from Diffusion.DiffusionRecommender import DiffusionAutoencoderRecommender_OptimizerMask

modelP = DiffusionAutoencoderRecommender_OptimizerMask(URM_train, use_gpu= False)
modelP.fit(epochs= 240, learning_rate= 0.00022443058798716912, l2_reg= 1.9501901504153858e-05, batch_size= 128, start_beta= 0.00038565911164407826, end_beta= 0.12444974448281955, noise_timesteps= 429.80432866068713, inference_timesteps= 1, objective= 'pred_x0', sgd_mode= 'rmsprop', activation_function= 'GELU', encoding_size= 245, next_layer_size_multiplier= 5, max_n_hidden_layers= 5, max_parameters= 1000000000.0)
result_df, _ = evaluator_validation.evaluateRecommender(modelP)
result_df.loc[10]

DiffusionAutoencoderRecommender: URM Detected 204 ( 5.3%) items with no interactions.
DiffusionAutoencoderRecommender: Architecture: [3883, 245]
fit!!!
429
DiffusionAutoencoderRecommender: Epoch 1, loss 2.08E+01
DiffusionAutoencoderRecommender: Epoch 1 of 240. Elapsed time 2.12 sec
DiffusionAutoencoderRecommender: Epoch 2, loss 2.02E+01
DiffusionAutoencoderRecommender: Epoch 2 of 240. Elapsed time 4.29 sec
DiffusionAutoencoderRecommender: Epoch 3, loss 1.97E+01
DiffusionAutoencoderRecommender: Epoch 3 of 240. Elapsed time 6.41 sec
DiffusionAutoencoderRecommender: Epoch 4, loss 1.97E+01
DiffusionAutoencoderRecommender: Epoch 4 of 240. Elapsed time 8.53 sec
DiffusionAutoencoderRecommender: Epoch 5, loss 1.93E+01
DiffusionAutoencoderRecommender: Epoch 5 of 240. Elapsed time 10.64 sec
DiffusionAutoencoderRecommender: Epoch 6, loss 1.97E+01
DiffusionAutoencoderRecommender: Epoch 6 of 240. Elapsed time 12.73 sec
DiffusionAutoencoderRecommender: Epoch 7, loss 1.97E+01
DiffusionAutoencoderReco

PRECISION                     0.178374
PRECISION_RECALL_MIN_DEN      0.234254
RECALL                        0.159779
MAP                           0.094984
MAP_MIN_DEN                   0.119691
MRR                           0.407674
NDCG                          0.210904
F1                            0.168565
HIT_RATE                      0.761777
ARHR_ALL_HITS                 0.612217
NOVELTY                       0.024251
AVERAGE_POPULARITY            0.474333
DIVERSITY_MEAN_INTER_LIST     0.942305
DIVERSITY_HERFINDAHL          0.994215
COVERAGE_ITEM                 0.210662
COVERAGE_ITEM_HIT             0.166109
ITEMS_IN_GT                   0.853206
COVERAGE_USER                 0.987583
COVERAGE_USER_HIT             0.752318
USERS_IN_GT                   0.987583
DIVERSITY_GINI                0.055612
SHANNON_ENTROPY               8.135414
RATIO_DIVERSITY_HERFINDAHL    0.995028
RATIO_DIVERSITY_GINI           0.15903
RATIO_SHANNON_ENTROPY         0.752376
RATIO_AVERAGE_POPULARITY 

In [ ]:
modelP.save_model(folder_path="/savedModels", file_name="/AutoencoderDiffusion0")


DiffusionAutoencoderRecommender: Saving model in file '/savedModels/AutoencoderDiffusion0'


OSError: [Errno 30] Read-only file system: '/savedModels'

In [ ]:
modelP2 = DiffusionAutoencoderRecommender_OptimizerMask(URM_train=URM_train, use_gpu=False)
modelP2.load_model(folder_path="savedModels", file_name="AutoencoderDiffusion0")

DiffusionAutoencoderRecommender: URM Detected 204 ( 5.3%) items with no interactions.
DiffusionAutoencoderRecommender: Loading model from file 'savedModelsAutoencoderDiffusion0'


AttributeError: 'DiffusionAutoencoderRecommender_OptimizerMask' object has no attribute 'use_batch_norm'

In [ ]:
modelP.inference_timesteps=100
result_df, _ = evaluator_validation.evaluateRecommender(modelP)
result_df.loc[10]

EvaluatorHoldout: Processed 5965 (100.0%) in 1.45 min. Users per second: 69


PRECISION                     0.060436
PRECISION_RECALL_MIN_DEN      0.080896
RECALL                        0.057029
MAP                           0.026853
MAP_MIN_DEN                   0.035202
MRR                           0.166286
NDCG                          0.066772
F1                            0.058683
HIT_RATE                      0.382733
ARHR_ALL_HITS                 0.209498
NOVELTY                       0.025672
AVERAGE_POPULARITY            0.348331
DIVERSITY_MEAN_INTER_LIST     0.920369
DIVERSITY_HERFINDAHL          0.992021
COVERAGE_ITEM                 0.168684
COVERAGE_ITEM_HIT              0.11692
ITEMS_IN_GT                   0.853206
COVERAGE_USER                 0.987583
COVERAGE_USER_HIT              0.37798
USERS_IN_GT                   0.987583
DIVERSITY_GINI                0.043977
SHANNON_ENTROPY               7.768874
RATIO_DIVERSITY_HERFINDAHL    0.992833
RATIO_DIVERSITY_GINI          0.125757
RATIO_SHANNON_ENTROPY         0.718478
RATIO_AVERAGE_POPULARITY 

In [ ]:
from RecommenderLuca import TwoLevelRec
import importlib

# Make some changes to your_module.py

importlib.reload(TwoLevelRec)
from RecommenderLuca.TwoLevelRec import TwoLevelRec


In [ ]:
inference_timesteps = {1,5,25,50}
for inference_timestep in inference_timesteps:
    modelP.inference_timesteps = inference_timestep
    two_level_rec1 = TwoLevelRec(URM_train=URM_train, recommender1= modelP, recommender2=itemKNNinstance)

    # Define the hyperparameter space
    itemknn_hyperparam_space = {
        "cutoff": list(range(10, 200)),
    }

    # Create an objective function
    objective = create_objective_d(recommender_instance=two_level_rec1, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=itemknn_hyperparam_space, evaluator_validation=evaluator_validation)

    # Create the study and run optimization
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=25)
    
    

[I 2023-07-24 19:59:34,588] A new study created in memory with name: no-name-2356a2e6-cb71-4e2d-a939-1f95846f1d08


TwoLevelRec: URM Detected 204 ( 5.3%) items with no interactions.
TwoLevelRec: URM Detected 204 ( 5.3%) items with no interactions.
TwoLevelRec: URM Detected 204 ( 5.3%) items with no interactions.
TwoLevelRec: URM Detected 204 ( 5.3%) items with no interactions.
EvaluatorHoldout: Processed 5965 (100.0%) in 2.92 sec. Users per second: 2039


[I 2023-07-24 20:00:04,701] Trial 0 finished with value: 0.08595536795327283 and parameters: {'cutoff': 77}. Best is trial 0 with value: 0.08595536795327283.


EvaluatorHoldout: Processed 5965 (100.0%) in 2.62 sec. Users per second: 2273


[I 2023-07-24 20:00:35,400] Trial 1 finished with value: 0.07576005534932663 and parameters: {'cutoff': 12}. Best is trial 0 with value: 0.08595536795327283.


...

[I 2023-07-24 21:01:23,220] Trial 147 finished with value: 0.08692632286219909 and parameters: {'cutoff': 150}. Best is trial 56 with value: 0.0872106401096344.


EvaluatorHoldout: Processed 5965 (100.0%) in 2.28 sec. Users per second: 2611


[I 2023-07-24 21:01:47,464] Trial 148 finished with value: 0.08531636929709051 and parameters: {'cutoff': 69}. Best is trial 56 with value: 0.0872106401096344.


EvaluatorHoldout: Processed 5965 (100.0%) in 2.33 sec. Users per second: 2561


[I 2023-07-24 21:02:11,753] Trial 149 finished with value: 0.08596806104392074 and parameters: {'cutoff': 101}. Best is trial 56 with value: 0.0872106401096344.


In [ ]:
inference_timesteps = {1,5,25,50}
for inference_timestep in inference_timesteps:
    modelP.inference_timesteps = inference_timestep
    two_level_rec1 = TwoLevelRec(URM_train=URM_train, recommender1= itemKNNinstance, recommender2=modelP)

    # Define the hyperparameter space
    two_levels_hyperparam_space = {
        "cutoff": list(range(10, 200)),
    }

    # Create an objective function
    objective = create_objective_d(recommender_instance=two_level_rec1, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=two_levels_hyperparam_space, evaluator_validation=evaluator_validation)

    # Create the study and run optimization
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=25)
    

    

[I 2023-07-24 21:57:32,427] A new study created in memory with name: no-name-b40b0caa-ca6d-4016-ad46-08d5bcf8890d


TwoLevelRec: URM Detected 204 ( 5.3%) items with no interactions.
EvaluatorHoldout: Processed 5965 (100.0%) in 3.30 sec. Users per second: 1807


[I 2023-07-24 21:57:36,526] Trial 0 finished with value: 0.09582460517037229 and parameters: {'cutoff': 52}. Best is trial 0 with value: 0.09582460517037229.


EvaluatorHoldout: Processed 5965 (100.0%) in 3.29 sec. Users per second: 1814


[I 2023-07-24 21:57:40,527] Trial 1 finished with value: 0.09518526058090167 and parameters: {'cutoff': 139}. Best is trial 0 with value: 0.09582460517037229.


...

[I 2023-07-24 22:30:35,875] Trial 22 finished with value: 0.0749872137734673 and parameters: {'cutoff': 94}. Best is trial 4 with value: 0.08779249324764866.


EvaluatorHoldout: Processed 5965 (100.0%) in 23.48 sec. Users per second: 254


[I 2023-07-24 22:30:59,984] Trial 23 finished with value: 0.08623322556180926 and parameters: {'cutoff': 16}. Best is trial 4 with value: 0.08779249324764866.


EvaluatorHoldout: Processed 5965 (100.0%) in 23.56 sec. Users per second: 253


[I 2023-07-24 22:31:24,197] Trial 24 finished with value: 0.08568667092430732 and parameters: {'cutoff': 19}. Best is trial 4 with value: 0.08779249324764866.


In [ ]:
study.best_params

{'topK': 93,
 'shrink': 74,
 'similarity': 'cosine',
 'normalize': True,
 'feature_weighting': 'TF-IDF'}

In [ ]:
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender


# Define the hyperparameter space
itemknn_hyperparam_space = {
    "num_factors": list(range(10, 160)),

}

svd_rec = PureSVDRecommender(URM_train=URM_train)
# Create an objective function
objective = create_objective_d(recommender_instance=svd_rec, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=itemknn_hyperparam_space, evaluator_validation=evaluator_validation)

# Create the study and run optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=150)


svd_rec.fit(**study.best_params)
result_df, _ = evaluator_validation.evaluateRecommender(svd_rec)
result_df.loc[10]

[I 2023-07-25 00:39:41,005] A new study created in memory with name: no-name-90c8932c-3c15-4873-bb75-3b61ce2802dc


PureSVDRecommender: URM Detected 204 ( 5.3%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.12 sec
EvaluatorHoldout: Processed 5965 (100.0%) in 1.89 sec. Users per second: 3156


[I 2023-07-25 00:39:43,032] Trial 0 finished with value: 0.08694459080083597 and parameters: {'num_factors': 11}. Best is trial 0 with value: 0.08694459080083597.


PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.51 sec
EvaluatorHoldout: Processed 5965 (100.0%) in 1.87 sec. Users per second: 3193


[I 2023-07-25 00:39:45,421] Trial 1 finished with value: 0.07611890791522005 and parameters: {'num_factors': 110}. Best is trial 0 with value: 0.08694459080083597.


...

[I 2023-07-25 00:45:14,819] Trial 148 finished with value: 0.08901548716720588 and parameters: {'num_factors': 52}. Best is trial 115 with value: 0.09580762782900284.


PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.34 sec
EvaluatorHoldout: Processed 5965 (100.0%) in 1.82 sec. Users per second: 3277


[I 2023-07-25 00:45:17,013] Trial 149 finished with value: 0.08464086137388784 and parameters: {'num_factors': 73}. Best is trial 115 with value: 0.09580762782900284.


PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.17 sec
EvaluatorHoldout: Processed 5965 (100.0%) in 1.82 sec. Users per second: 3283


PRECISION                     0.178927
PRECISION_RECALL_MIN_DEN      0.232087
RECALL                        0.156279
MAP                           0.095544
MAP_MIN_DEN                   0.118085
MRR                            0.40377
NDCG                          0.208653
F1                            0.166838
HIT_RATE                      0.751215
ARHR_ALL_HITS                 0.610106
NOVELTY                       0.024088
AVERAGE_POPULARITY            0.481357
DIVERSITY_MEAN_INTER_LIST     0.941094
DIVERSITY_HERFINDAHL          0.994094
COVERAGE_ITEM                 0.163533
COVERAGE_ITEM_HIT              0.13881
ITEMS_IN_GT                   0.853206
COVERAGE_USER                 0.987583
COVERAGE_USER_HIT             0.741887
USERS_IN_GT                   0.987583
DIVERSITY_GINI                0.047717
SHANNON_ENTROPY                7.95239
RATIO_DIVERSITY_HERFINDAHL    0.994906
RATIO_DIVERSITY_GINI          0.136453
RATIO_SHANNON_ENTROPY          0.73545
RATIO_AVERAGE_POPULARITY 

In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

slim_rec = SLIMElasticNetRecommender(URM_train)
slim_rec.fit()
result_df, _ = evaluator_validation.evaluateRecommender(slim_rec)
result_df.loc[10]

SLIMElasticNetRecommender: URM Detected 204 ( 5.3%) items with no interactions.
SLIMElasticNetRecommender: Processed 3883 (100.0%) in 1.32 min. Items per second: 48.96
EvaluatorHoldout: Processed 5965 (100.0%) in 2.05 sec. Users per second: 2912


PRECISION                     0.180436
PRECISION_RECALL_MIN_DEN      0.236134
RECALL                        0.160031
MAP                           0.097529
MAP_MIN_DEN                   0.121712
MRR                           0.413098
NDCG                          0.217846
F1                            0.169622
HIT_RATE                      0.761274
ARHR_ALL_HITS                  0.62356
NOVELTY                        0.02383
AVERAGE_POPULARITY            0.519913
DIVERSITY_MEAN_INTER_LIST     0.920168
DIVERSITY_HERFINDAHL          0.992001
COVERAGE_ITEM                 0.180273
COVERAGE_ITEM_HIT             0.125934
ITEMS_IN_GT                   0.853206
COVERAGE_USER                 0.987583
COVERAGE_USER_HIT             0.751821
USERS_IN_GT                   0.987583
DIVERSITY_GINI                0.041659
SHANNON_ENTROPY               7.702668
RATIO_DIVERSITY_HERFINDAHL    0.992813
RATIO_DIVERSITY_GINI          0.119129
RATIO_SHANNON_ENTROPY         0.712355
RATIO_AVERAGE_POPULARITY 

In [ ]:
recommenders_instances = {itemKNNinstance, slim_rec, svd_rec}
recommenders_instances
for traditional_rec in recommenders_instances:
    print(traditional_rec.RECOMMENDER_NAME)


PureSVDRecommender
ItemKNNCFRecommender
SLIMElasticNetRecommender


In [ ]:
inference_timesteps = {1,5,25,50}
recommenders_instances = {itemKNNinstance, slim_rec, svd_rec}
print(recommenders_instance for recommenders_instance in recommenders_instances)
for traditional_rec in recommenders_instances:
    for inference_timestep in inference_timesteps:
        modelP.inference_timesteps = inference_timestep
        print(traditional_rec.RECOMMENDER_NAME)
        two_level_rec1 = TwoLevelRec(URM_train=URM_train, recommender1= modelP, recommender2=traditional_rec)
        print(two_level_rec1.RECOMMENDER_NAME)

        # Define the hyperparameter space
        itemknn_hyperparam_space = {
            "cutoff": list(range(10, 200)),
        }

        # Create an objective function
        objective = create_objective_d(recommender_instance=two_level_rec1, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=itemknn_hyperparam_space, evaluator_validation=evaluator_validation)

        # Create the study and run optimization
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=50)
        
        two_level_rec1.fit(** study.best_params)
        filename = "best_models" + str(inference_timestep) + "_timesteps" + ".csv"
        result_df, _ = evaluator_validation.evaluateRecommender(two_level_rec1)
        result_df["RECOMMENDER"] = two_level_rec1.RECOMMENDER_NAME
        result_df["PARAMS"] = str(study.best_params)

        
        # Check if file exists
        if os.path.isfile(filename):
            # If it exists, append without writing the header
            pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
            # If it doesn't exist, create it, write the header
            pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

        

[I 2023-07-25 01:38:10,423] A new study created in memory with name: no-name-89f93437-a031-4219-8dac-84c0b97c5633


<generator object <genexpr> at 0x7fe058b57b30>
PureSVDRecommender
Recommender_Base_Class: URM Detected 204 ( 5.3%) items with no interactions.
DiffusionAutoencoderRecommender_PureSVDRecommender_TwoLevelRec
EvaluatorHoldout: Processed 5965 (100.0%) in 2.23 sec. Users per second: 2674


[I 2023-07-25 01:38:14,476] Trial 0 finished with value: 0.09566391250548849 and parameters: {'cutoff': 70}. Best is trial 0 with value: 0.09566391250548849.


EvaluatorHoldout: Processed 5965 (100.0%) in 2.21 sec. Users per second: 2699


[I 2023-07-25 01:38:18,183] Trial 1 finished with value: 0.0955612368445563 and parameters: {'cutoff': 118}. Best is trial 0 with value: 0.09566391250548849.


...

EvaluatorHoldout: Processed 5965 (100.0%) in 2.25 sec. Users per second: 2651


[I 2023-07-25 05:02:54,725] Trial 48 finished with value: 0.0942738394603442 and parameters: {'cutoff': 92}. Best is trial 2 with value: 0.09651572665948188.


EvaluatorHoldout: Processed 5965 (100.0%) in 2.28 sec. Users per second: 2616


[I 2023-07-25 05:03:18,921] Trial 49 finished with value: 0.09476720685480133 and parameters: {'cutoff': 97}. Best is trial 2 with value: 0.09651572665948188.


EvaluatorHoldout: Processed 5965 (100.0%) in 2.31 sec. Users per second: 2577


In [ ]:
inference_timesteps = {1,5,25,50}
recommenders_instances = {itemKNNinstance, slim_rec, svd_rec}
for traditional_rec in recommenders_instances:
    for inference_timestep in inference_timesteps:
        modelP.inference_timesteps = inference_timestep
        two_level_rec1 = TwoLevelRec(URM_train=URM_train, recommender1= traditional_rec, recommender2=modelP)
        print(two_level_rec1.RECOMMENDER_NAME)
        # Define the hyperparameter space
        two_levels_hyperparam_space = {
            "cutoff": list(range(10, 200)),
        }

        # Create an objective function
        objective = create_objective_d(recommender_instance=two_level_rec1, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=two_levels_hyperparam_space, evaluator_validation=evaluator_validation)

        # Create the study and run optimization
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=50)
        
        filename = "best_models" + str(inference_timestep) + "_timesteps" + ".csv"
        result_df, _ = evaluator_validation.evaluateRecommender(two_level_rec1)
        result_df["RECOMMENDER"] = two_level_rec1.RECOMMENDER_NAME
        result_df["PARAMS"] = str(study.best_params)

        
        # Check if file exists
        if os.path.isfile(filename):
            # If it exists, append without writing the header
            pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
            # If it doesn't exist, create it, write the header
            pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

    

    

[I 2023-07-25 05:03:43,195] A new study created in memory with name: no-name-3c9af4c2-9978-4866-abab-cd8035ef09be


Recommender_Base_Class: URM Detected 204 ( 5.3%) items with no interactions.
PureSVDRecommender_DiffusionAutoencoderRecommender_TwoLevelRec
EvaluatorHoldout: Processed 5965 (100.0%) in 3.11 sec. Users per second: 1918


[I 2023-07-25 05:03:46,672] Trial 0 finished with value: 0.09580343671416612 and parameters: {'cutoff': 36}. Best is trial 0 with value: 0.09580343671416612.


EvaluatorHoldout: Processed 5965 (100.0%) in 3.08 sec. Users per second: 1937


[I 2023-07-25 05:03:50,110] Trial 1 finished with value: 0.09515458561183629 and parameters: {'cutoff': 51}. Best is trial 0 with value: 0.09580343671416612.


...

EvaluatorHoldout: Processed 5965 (100.0%) in 23.54 sec. Users per second: 253


[I 2023-07-25 08:27:15,406] Trial 48 finished with value: 0.0951572266794399 and parameters: {'cutoff': 10}. Best is trial 48 with value: 0.0951572266794399.


EvaluatorHoldout: Processed 5965 (100.0%) in 23.49 sec. Users per second: 254


[I 2023-07-25 08:27:39,489] Trial 49 finished with value: 0.07521622161018675 and parameters: {'cutoff': 103}. Best is trial 48 with value: 0.0951572266794399.


EvaluatorHoldout: Processed 5965 (100.0%) in 23.77 sec. Users per second: 251


In [ ]:
OTTIMIZZA TWOLEVELS, RECOMMEND PER OGNI RECOMMENDER, COMPUT ITEM SCORE PER OGNI RECOMMENDER, PRODOTTO CARTESIANO 


In [ ]:
one_timestep_models = pd.read_csv("best_models1_timesteps.csv")
five_timestep_models = pd.read_csv("best_models5_timesteps.csv")
twenty_timestep_models = pd.read_csv("best_models25_timesteps.csv")
fifty_timestep_models = pd.read_csv("best_models50_timesteps.csv")

In [ ]:
one_timestep_models["AVERAGE_POPULARITY"]

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY,RECOMMENDER,PARAMS
0,0.180721,0.235964,0.160426,0.096488,0.119901,0.406777,0.211023,0.169970,0.759933,0.616304,...,0.987583,0.056724,8.192917,0.995638,0.162209,0.757694,1.674852,0.065874,DiffusionAutoencoderRecommender_PureSVDRecomme...,{'cutoff': 13}
1,0.179581,0.235790,0.160184,0.093786,0.117072,0.397288,0.201674,0.169329,0.760771,0.599643,...,0.987583,0.059448,8.235371,0.995443,0.169998,0.761620,1.686815,0.065955,DiffusionAutoencoderRecommender_ItemKNNCFRecom...,{'cutoff': 13}
2,0.184761,0.241991,0.163945,0.100365,0.125150,0.416621,0.220698,0.173731,0.765968,0.635297,...,0.987583,0.048152,7.935934,0.994302,0.137697,0.733928,1.796585,0.065110,DiffusionAutoencoderRecommender_SLIMElasticNet...,{'cutoff': 20}
3,0.178189,0.233575,0.158938,0.095191,0.120165,0.409657,0.211265,0.168014,0.761609,0.613939,...,0.987583,0.056370,8.161026,0.995208,0.161198,0.754745,1.701651,0.065820,PureSVDRecommender_DiffusionAutoencoderRecomme...,{'cutoff': 22}
4,0.180151,0.235860,0.160694,0.096075,0.120982,0.408076,0.211822,0.169867,0.762615,0.615499,...,0.987583,0.057172,8.177789,0.995205,0.163492,0.756295,1.704953,0.065821,ItemKNNCFRecommender_DiffusionAutoencoderRecom...,{'cutoff': 42}
5,0.178340,0.233080,0.158249,0.095295,0.119996,0.408006,0.210557,0.167695,0.759095,0.613132,...,0.987583,0.055475,8.133980,0.995061,0.158637,0.752243,1.716040,0.065728,SLIMElasticNetRecommender_DiffusionAutoencoder...,{'cutoff': 16}


In [ ]:
one_timestep_models[["RECOMMENDER","PARAMS","MAP","PRECISION", "NDCG", "RATIO_NOVELTY","DIVERSITY_MEAN_INTER_LIST", "DIVERSITY_GINI","COVERAGE_ITEM", "AVERAGE_POPULARITY"]]

,RECOMMENDER,PARAMS,MAP,PRECISION,NDCG,RATIO_NOVELTY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_GINI,COVERAGE_ITEM,AVERAGE_POPULARITY
0,DiffusionAutoencoderRecommender_PureSVDRecomme...,{'cutoff': 13},0.096488,0.180721,0.211023,0.065874,0.948406,0.056724,0.204996,0.462654
1,DiffusionAutoencoderRecommender_ItemKNNCFRecom...,{'cutoff': 13},0.093786,0.179581,0.201674,0.065955,0.946452,0.059448,0.221736,0.465959
2,DiffusionAutoencoderRecommender_SLIMElasticNet...,{'cutoff': 20},0.100365,0.184761,0.220698,0.065110,0.935050,0.048152,0.185424,0.496281
3,PureSVDRecommender_DiffusionAutoencoderRecomme...,{'cutoff': 22},0.095191,0.178189,0.211265,0.065820,0.944104,0.056370,0.210662,0.470057
4,ItemKNNCFRecommender_DiffusionAutoencoderRecom...,{'cutoff': 42},0.096075,0.180151,0.211822,0.065821,0.944077,0.057172,0.224569,0.470969
5,SLIMElasticNetRecommender_DiffusionAutoencoder...,{'cutoff': 16},0.095295,0.178340,0.210557,0.065728,0.942636,0.055475,0.204739,0.474032


In [ ]:
five_timestep_models[["RECOMMENDER","PARAMS","MAP","PRECISION", "NDCG", "RATIO_NOVELTY","DIVERSITY_MEAN_INTER_LIST", "DIVERSITY_GINI","COVERAGE_ITEM", "AVERAGE_POPULARITY"]]

,RECOMMENDER,PARAMS,MAP,PRECISION,NDCG,RATIO_NOVELTY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_GINI,COVERAGE_ITEM,AVERAGE_POPULARITY
0,DiffusionAutoencoderRecommender_PureSVDRecomme...,{'cutoff': 178},0.095931,0.179296,0.209406,0.065354,0.942334,0.048672,0.166366,0.479033
1,DiffusionAutoencoderRecommender_ItemKNNCFRecom...,{'cutoff': 15},0.089773,0.172741,0.191912,0.066148,0.948080,0.060586,0.221221,0.459491
2,DiffusionAutoencoderRecommender_SLIMElasticNet...,{'cutoff': 37},0.099378,0.183085,0.218716,0.065000,0.932713,0.046905,0.182848,0.501109
3,PureSVDRecommender_DiffusionAutoencoderRecomme...,{'cutoff': 12},0.094864,0.179447,0.207514,0.065627,0.946691,0.052534,0.175637,0.468937
4,ItemKNNCFRecommender_DiffusionAutoencoderRecom...,{'cutoff': 34},0.091535,0.172607,0.201418,0.065928,0.943926,0.056841,0.213752,0.468033
5,SLIMElasticNetRecommender_DiffusionAutoencoder...,{'cutoff': 18},0.091172,0.171769,0.199946,0.065817,0.942483,0.055193,0.199330,0.471577


In [ ]:
twenty_timestep_models[["RECOMMENDER","PARAMS","MAP","PRECISION", "NDCG", "RATIO_NOVELTY","DIVERSITY_MEAN_INTER_LIST", "DIVERSITY_GINI","COVERAGE_ITEM", "AVERAGE_POPULARITY"]]

,RECOMMENDER,PARAMS,MAP,PRECISION,NDCG,RATIO_NOVELTY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_GINI,COVERAGE_ITEM,AVERAGE_POPULARITY
0,DiffusionAutoencoderRecommender_PureSVDRecomme...,{'cutoff': 183},0.093628,0.175926,0.206198,0.065621,0.946414,0.052778,0.178213,0.469196
1,DiffusionAutoencoderRecommender_ItemKNNCFRecom...,{'cutoff': 169},0.083405,0.161023,0.175489,0.067359,0.957020,0.079874,0.318826,0.420642
2,DiffusionAutoencoderRecommender_SLIMElasticNet...,{'cutoff': 178},0.096221,0.178826,0.214674,0.065018,0.931743,0.047013,0.185681,0.500793
3,PureSVDRecommender_DiffusionAutoencoderRecomme...,{'cutoff': 18},0.072345,0.144761,0.167674,0.067290,0.956352,0.065937,0.211434,0.419393
4,ItemKNNCFRecommender_DiffusionAutoencoderRecom...,{'cutoff': 15},0.077298,0.154099,0.174915,0.067045,0.955826,0.067058,0.232037,0.427453
5,SLIMElasticNetRecommender_DiffusionAutoencoder...,{'cutoff': 10},0.074930,0.149992,0.171637,0.066818,0.953447,0.062175,0.195725,0.433092


In [ ]:
fifty_timestep_models[["RECOMMENDER","PARAMS","MAP","PRECISION", "NDCG", "RATIO_NOVELTY","DIVERSITY_MEAN_INTER_LIST", "DIVERSITY_GINI","COVERAGE_ITEM", "AVERAGE_POPULARITY"]]

,RECOMMENDER,PARAMS,MAP,PRECISION,NDCG,RATIO_NOVELTY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_GINI,COVERAGE_ITEM,AVERAGE_POPULARITY
0,DiffusionAutoencoderRecommender_PureSVDRecomme...,{'cutoff': 197},0.080997,0.153412,0.175976,0.066598,0.957797,0.063390,0.193922,0.428622
1,DiffusionAutoencoderRecommender_ItemKNNCFRecom...,{'cutoff': 197},0.072604,0.141174,0.150886,0.067896,0.962150,0.082741,0.302086,0.394895
2,DiffusionAutoencoderRecommender_SLIMElasticNet...,{'cutoff': 197},0.083659,0.156798,0.183967,0.065972,0.947942,0.056376,0.192119,0.457588
3,PureSVDRecommender_DiffusionAutoencoderRecomme...,{'cutoff': 12},0.045456,0.098826,0.104008,0.068728,0.959673,0.066808,0.203193,0.365793
4,ItemKNNCFRecommender_DiffusionAutoencoderRecom...,{'cutoff': 15},0.058132,0.125834,0.127596,0.068127,0.963596,0.075913,0.255473,0.383433
5,SLIMElasticNetRecommender_DiffusionAutoencoder...,{'cutoff': 13},0.043701,0.095373,0.100439,0.068220,0.955258,0.061891,0.188772,0.381021
